In [ ]:
#Carregando bibs

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


In [ ]:

#Carregamento de dados
# Observe que demo01.csv está na mesma pasta que este notebook
# do contrário, seria necessário indicar a pasta no nome do arquivo

df = pd.read_csv('demo01.csv')
print ("Número de linhas e colunas da tabela: {}".format(df.shape))

df.head()



In [ ]:
# Interpretando dados - Univariada
print(df['mau'].value_counts())
print("\nTaxa de inadimplentes:")
print(df['mau'].mean())
print()

#Gráfico de barras com os mesmos dados anteriores
var = 'mau'
grafico_barras = df[var].value_counts().plot.bar()



In [ ]:
'''Tarefa 01 - gráfico de barras
Com base no código da célula anterior, construa um gráfico de barras para pelo menos duas outras variáveis.
'''
plt.clf()
var_pess_red = "qt_pessoas_residencia"
var_est_civi = "sexo"

sns.displot(df, x = var_pess_red, bins = 50)
plt.show()
sns.displot(df, x = var_est_civi, bins = 50)
plt.show()


In [ ]:
# Alterando valores de -1000 pra -2, para visualizar melhor no gráfico
# Mostrar tempo emprego
var_temp_empr = "tempo_emprego"
df.loc[df[var]<0,var_temp_empr] = -2

plt.clf()
var_temp_empr = "tempo_emprego"

sns.displot(df, x = var_temp_empr, bins = 50)
plt.show()

In [ ]:
'''
Tarefa 02 - Histograma
Com base no código da célula anterior, construa o histograma da variavel `idade`
'''
var_idade = "idade"
df.loc[df[var_idade]<0,var_idade] = -2
plt.clf()
var_idade = "idade"

sns.displot(df, x = var_idade, bins = 50)
plt.show()

In [ ]:
'''Entendimento dos dados - Bivariadas
Entender a alteração da inadimplência indicada pela variável resposta (```AtrasoRelevante2anos```) e as variáveis explicativas (demais). Para isto, vamos calcular a taxa de inadimplentes (qtd inadimplentes / total) para diferentes grupos definidos pelas variáveis explicativas.
'''
var_idade = 'idade'
cat_srs, bins = pd.qcut(df[var_idade], 4, retbins=True)
g = df.groupby(cat_srs)
biv = g['mau'].mean()

ax = biv.plot.line()
ax.set_ylabel("Proporção de inadimplentes")
ticks = plt.xticks(range(len(biv.index.values)), biv.index.values, rotation = 90)

In [ ]:
'''Tarefa 03 - Bivariada
Com base no código da célula anterior, construa uma análise bivariada para a variável tempo_emprego. Em seguida, insira uma célula de markdown e conclua se a variável parece discriminar risco de crédito.
'''
var_temp_empr = 'tempo_emprego'
df.loc[df[var_temp_empr]<0,var_temp_empr] = -2
cat_srs, bins = pd.qcut(df[var_temp_empr], 4, retbins=True)
g = df.groupby(cat_srs)
biv = g['mau'].mean()

ax = biv.plot.line()
ax.set_ylabel("Proporção de inadimplentes")
ticks = plt.xticks(range(len(biv.index.values)), biv.index.values, rotation = 90)


## Conforme o gráfico mostra, a inadimplência é inversamente proporcional ao tempo de emprego

In [ ]:
'''
## Etapa 3 Crisp-DM: Preparação dos dados
Os dados já estão pré-selecionados, construídos e integrados, mas há dados faltantes que serão eliminados na próxima célula

'''
metadata = pd.DataFrame(df.dtypes, columns = ['tipo'])

metadata['n_categorias'] = 0

for var in metadata.index:
    metadata.loc[var,'n_categorias'] = len(df.groupby([var]).size())

metadata

In [ ]:
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

In [ ]:
for var in metadata[metadata['tipo'] == 'object'].index:
    df = convert_dummy(df, var)

In [ ]:
df

In [ ]:
'''
Etapa 4 Crisp-DM: Modelagem
Dividindo a base em treino e teste
'''
# Tirando a v. resposta da base de treinamento
x = df.drop("mau",axis = 1)
y = df["mau"]

# Tirando ID da base de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train

In [ ]:
# Treinar uma Random Forest com 5 árvores

clf = RandomForestClassifier(n_estimators=3)
clf.fit(x_train,y_train)

In [ ]:
# Calculando a acuracia

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}%'.format(acc*100))

In [ ]:
# Matriz de confusão

tab = pd.crosstab(index = y_pred, columns = y_test)
print(tab[1][0]/(tab[1][0] + tab[0][0]))
print(tab[1][1]/(tab[1][1] + tab[0][1]))
tab

In [ ]:
'''
Tarefa 04 - Bivariada
Para essa tarefa, crie três células extras, e altere o parâmetro n_estimators= da função RandomForestClassifier e insira uma quantidade maior que 3 nesse parâmetro. Rode as três células anteriores para calcular a acurácia do modelo e veja se você consegue uma acurácia melhor.
'''
# Treinar uma Random Forest com 5 árvores

clf = RandomForestClassifier(n_estimators=99)
clf.fit(x_train,y_train)

# Calculando a acuracia

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}%'.format(acc*100))

# Matriz de confusão

tab = pd.crosstab(index = y_pred, columns = y_test)
print(tab[1][0]/(tab[1][0] + tab[0][0]))
print(tab[1][1]/(tab[1][1] + tab[0][1]))
tab

#Etapa 5 Crisp-DM: Avaliação dos resultados

#Etapa 6 Crisp-DM: Implantação